In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [4]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../../creds/'
creds_orch_file = "creds-orch.txt"

with open(os.path.join(path, creds_orch_file), "r") as myfile:
    usr_name = myfile.readline().strip()  # Read the first line and strip any leading/trailing whitespace
    tabExpTrackAPIKey = myfile.readline().strip()  # Read the second line and strip any leading/trailing whitespace

print('API Key Loaded!')

API Key Loaded!


In [7]:
from requests.exceptions import RequestsDependencyWarning

expname = "LCN_experiment_2"

slice_start = 0
slice_size = 1000
i = 0
data = []
while True:
    url = ""
    payload = json.dumps({"experiment": expname, "slice_start": slice_start, "slice_end": slice_start+slice_size})
    headers = {'Name': usr_name,'Seceret': tabExpTrackAPIKey,'Content-Type': 'application/json'}
    t1 = time.time()
    resp_unparse = requests.request("POST", url, headers=headers, data=payload)
    t2=time.time()
    resp = json.loads(resp_unparse.text)
    t3=time.time()
    print('========= {} ========'.format(i))
    print('request time: {}'.format(t2-t1))
    print('parse time: {}'.format(t3-t2))
    print('length: {}'.format(len(resp)))
    data.extend(resp)
    if len(resp) == 0:
        break
    slice_start+=slice_size
    i+=1

print('total length: {}'.format(len(data)))

========= 0 ========
request time: 2.4080469608306885
parse time: 0.043786048889160156
length: 1000
========= 1 ========
request time: 2.536015033721924
parse time: 0.19631481170654297
length: 1000
========= 2 ========
request time: 2.181389093399048
parse time: 0.043688058853149414
length: 1000
========= 3 ========
request time: 2.383979082107544
parse time: 0.01822495460510254
length: 1000
========= 4 ========
request time: 2.1603291034698486
parse time: 0.04429793357849121
length: 1000
========= 5 ========
request time: 2.305868148803711
parse time: 0.029631853103637695
length: 1000
========= 6 ========
request time: 2.448884963989258
parse time: 0.04821300506591797
length: 1000
========= 7 ========
request time: 1.7836289405822754
parse time: 0.017457246780395508
length: 126
========= 8 ========
request time: 1.6168668270111084
parse time: 0.0002741813659667969
length: 0
total length: 7126


In [12]:
data[3]

{'_id': '650b4d4d412e46634033ea0e',
 'metrics_per_epoch': [{'train_loss': 1219.076370716095, 'epoch': 1},
  {'train_loss': 415.542289853096, 'epoch': 2},
  {'train_loss': 368.49646723270416, 'epoch': 3},
  {'train_loss': 336.4351638555527, 'epoch': 4},
  {'train_loss': 341.36901116371155, 'epoch': 5},
  {'train_loss': 335.69392907619476, 'epoch': 6},
  {'train_loss': 328.47513139247894, 'epoch': 7},
  {'train_loss': 326.9390068054199, 'epoch': 8},
  {'train_loss': 337.7342299222946, 'epoch': 9},
  {'train_loss': 323.3105516433716, 'epoch': 10},
  {'train_loss': 321.0703556537628, 'epoch': 11},
  {'train_loss': 321.2774603366852, 'epoch': 12},
  {'train_loss': 320.62992763519287, 'epoch': 13},
  {'train_loss': 321.01994371414185, 'epoch': 14},
  {'train_loss': 321.34505212306976, 'epoch': 15},
  {'train_loss': 320.64058953523636, 'epoch': 16},
  {'train_loss': 320.45185828208923, 'epoch': 17},
  {'train_loss': 320.3091918230057, 'epoch': 18},
  {'train_loss': 321.3091276884079, 'epoch':

In [15]:
df_model = [] # model
df_dataset = [] # task
df_hyperparameters = [] # hyp
df_metrics = []




for observation in data:
    df_model.append(observation['model'])
    df_dataset.append(observation['task'])
    df_hyperparameters.append(observation['hyp'])
    df_metrics.append(observation['metrics_per_epoch'][-1])
else:
    data_df = pd.DataFrame({
        'model': df_model,
        'dataset': df_dataset,
        'hyperparameters': df_hyperparameters,
        'metrics': df_metrics
    })

In [16]:
data_df

,model,dataset,hyperparameters,metrics
0,LCN_reg,336-361072,"{'depth': 8, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': None, 'epoch': 1}"
1,LLN_reg,336-361072,"{'depth': 9, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': None, 'epoch': 1}"
2,LCN_reg,336-361072,"{'depth': 6, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': None, 'epoch': 1}"
3,LLN_reg,336-361072,"{'depth': 9, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': 319.6605763435364, 'train_metri..."
4,LLN_reg,336-361072,"{'depth': 5, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': None, 'epoch': 1}"
...,...,...,...,...
7121,LCN_cls,337-361277,"{'depth': 2, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': 0.010059343594312669, 'train_me..."
7122,LLN_cls,337-361277,"{'depth': 2, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': 0.005389809650182724, 'train_me..."
7123,LCN_cls,337-361277,"{'depth': 9, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': 0.009550702953338623, 'train_me..."
7124,LLN_cls,337-361277,"{'depth': 5, 'seed': 42, 'drop_type': 'isDropC...","{'train_loss': 0.005012368750572205, 'train_me..."


In [17]:
data_df.to_csv('LcnTotalResults.csv', index=False)